### Load data from database

In [1]:
# imports
import pandas as pd
import psycopg2
import config as c

# import for multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# connection to database
connection = psycopg2.connect(
    host = c.host,
    port = c.port,
    user = c.user,
    password = c.password,
    database = c.database
    )
cursor=connection.cursor()

In [5]:
data_sql = """
SELECT *
FROM comprehensive_dataset;
"""

# load in tables as dataframes
data_df = pd.read_sql(data_sql, con=connection)
data_df.head()

,GUID,Name,HallOfFameStatus,HallofFameClass,YearDrafted,TO_YEAR,Years_Played,HOF_Elgibility_Year,Pick,Team,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,AGE_ROOKIE_SEASON
0,MIKENILES1980,Mike Niles,Not Inducted,NaN,1980,1980,1,1985,NaN,None,...,45.9,0.6,0.7,1.3,0.3,0.2,0.0,0.6,1.4,NaN
1,JAREDSULLINGER2012,Jared Sullinger,Not Inducted,NaN,2012,2016,5,2021,21.0,BOS,...,74.6,2.0,3.8,5.9,0.8,0.5,0.5,0.6,10.1,20.0
2,ARNETTMOULTRIE2012,Arnett Moultrie,Not Inducted,NaN,2012,2013,2,2018,27.0,MIA,...,64.3,1.6,1.5,3.1,0.2,0.4,0.2,0.4,5.8,21.0
3,PERRYJONESIII2012,Perry Jones III,Not Inducted,NaN,2012,2014,3,2019,NaN,None,...,66.7,0.3,1.3,1.6,0.3,0.1,0.2,0.4,2.4,21.0
4,FESTUSEZELI2012,Festus Ezeli,Not Inducted,NaN,2012,2016,5,2021,30.0,GSW,...,53.1,1.8,2.2,4.0,0.3,0.3,0.9,0.8,5.5,22.0


## Preprocessing 

In [4]:
# get_dummies to encode HallofFameClass column
data_df = pd.get_dummies(data_df, prefix='HOF', columns=['HallOfFameStatus'])
data_df.drop(['HOF_Not Inducted'], axis=1, inplace=True) # remove extra get_dummies column, drop_first no ideal

# drop features
data_df.drop(['GUID', 'Name', 'Team', 'College', # dropping object columns  
              
              'HallofFameClass', 'YearDrafted', 'TO_YEAR', 'Years_Played', 
              'HOF_Elgibility_Year', 'GP', 'MIN', 'AGE_ROOKIE_SEASON' # non performance stats
                ], axis=1, inplace=True)

# fill NaNs with 0
data_df = data_df.fillna(0)

data_df.head()
print(data_df.shape)

,Pick,PTS,FGM,FGA,FG%,3P_Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,HOF_Hall of Fame Member
0,0.0,2.6,1.1,3.1,34.8,0.0,0.1,50.0,0.4,0.8,45.9,0.6,0.7,1.3,0.3,0.2,0.0,0.6,1.4,0
1,21.0,6.0,2.5,5.0,49.3,0.0,0.1,20.0,1.0,1.4,74.6,2.0,3.8,5.9,0.8,0.5,0.5,0.6,10.1,0
2,27.0,3.7,1.7,2.9,58.2,0.0,0.0,0.0,0.4,0.6,64.3,1.6,1.5,3.1,0.2,0.4,0.2,0.4,5.8,0
3,0.0,2.3,1.0,2.6,39.4,0.0,0.1,0.0,0.3,0.4,66.7,0.3,1.3,1.6,0.3,0.1,0.2,0.4,2.4,0
4,30.0,2.4,0.9,2.2,43.8,0.0,0.0,0.0,0.6,1.0,53.1,1.8,2.2,4.0,0.3,0.3,0.9,0.8,5.5,0


(1217, 20)


### Prep Model

In [6]:
# ML imports
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SVMSMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

In [7]:
# create our features
X = data_df.drop(['HOF_Hall of Fame Member'], axis=1)

# create our target
y = data_df['HOF_Hall of Fame Member']

KeyError: "['HOF_Hall of Fame Member'] not found in axis"

In [8]:
# normal train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)
Counter(y_train)

NameError: name 'X' is not defined

In [ ]:
## implement SVM SMOTE resampling
X_resampled, y_resampled = SVMSMOTE(random_state=2, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

In [ ]:
# logistic regression using SVM SMOTE data
model = LogisticRegression(solver='liblinear', random_state=2)
model.fit(X_resampled, y_resampled)

## Save Model

In [ ]:
# import 
import pickle

In [ ]:
# open file and load model
pickle.dump(model, open('ml_model.sav', 'wb'))